In [1]:
from func_fix_curvefit_params import *

'/Users/f0053cz/Documents/psyanim_v2/scripts_for_beh_paper/code'

In [2]:
import pandas as pd
import numpy as np

In [3]:
def fix_obj_center(df):
    for i,row in df.iterrows():
        if row['flipped'] == False: # only deal with regular curves
            if np.isnan(row['obj_center']): # full curve is in the upper or lower half 
                #(not fixed in the curvefit script)
                if (row['bias_xmin'] >= 0.5) & (row['bias_xmax'] <= 0.5): # upper half
                    df.at[i, 'obj_center'] = 0
                elif (row['bias_xmax'] >= 0.5) & (row['bias_xmin'] <= 0.5): # lower half
                    df.at[i, 'obj_center'] = 1

            if np.isinf(row['obj_center']):
                if (row['bias_xmin'] >= 0.5) & (row['bias_xmax'] <= 0.5): # upper half
                    df.at[i, 'obj_center'] = 0
                elif (row['bias_xmax'] >= 0.5) & (row['bias_xmin'] <= 0.5): # lower half
                    df.at[i, 'obj_center'] = 1

            if row['obj_center'] > 1:
                df.at[i, 'obj_center'] = 1
                
            if row['obj_center'] < 0:
                df.at[i, 'obj_center'] = 0

        elif row['flipped'] == True: # if flipped, assign obj_center as np.nan
            df.at[i, 'obj_center'] = np.nan
             
    return df

def add_new_params(df):

    df['range'] = np.nan #1-df['bias_xmin']-df['bias_xmax']
    df['bias'] =  np.nan #df['bias_xmin']-df['bias_xmax']

    for i,row in df.iterrows():
        
        if (row['bias_xmin'] <= 0) & (row['bias_xmax'] <= 0):
            df.at[i, 'bias_xmin'] = 0
            df.at[i, 'bias'] = 0 # no fraction of bias comes from either end
            df.at[i,'range']= 1

        elif (row['bias_xmin'] < 0) | (row['bias_xmax'] < 0):
            if (row['bias_xmin'] < 0) & (row['bias_xmax'] >= 0):
                df.at[i, 'bias_xmin'] = 0
                df.at[i, 'bias'] = 0 # no fraction of bias comes from the lower end
                df.at[i,'range'] = 1 - df.at[i,'bias_xmax']
    
            elif (row['bias_xmin'] >= 0) & (row['bias_xmax'] < 0):
                df.at[i, 'bias_xmax'] = 0
                df.at[i, 'bias'] = 1 # no fraction of bias comes from the upper end
                df.at[i,'range'] = 1 - df.at[i,'bias_xmin']

        elif (row['bias_xmin'] >= 0) & (row['bias_xmax'] >= 0):
                if (row['bias_xmin'] == 0) & (row['bias_xmax'] == 0):
                    df.at[i,'range'] = 1
                    df.at[i,'bias'] =  0
                else:
                    df.at[i,'range'] = 1-df.at[i,'bias_xmin']-df.at[i,'bias_xmax']
                    df.at[i,'bias'] =  df.at[i,'bias_xmin']/(df.at[i,'bias_xmin'] + df.at[i,'bias_xmax'])
                 #df.at[i,'bias_xmin']-df.at[i,'bias_xmax'] (bias_diff)
    return df            

def get_p(p):
    if p < .001:
        return "p < .001"
    elif p < .01:
        return f"p={p:.3f}"
    else:
        return f"p={p:.2f}"



In [4]:
traits_root_folder = '/Users/f0053cz/Documents/psyanim_v2/subtlety_playfight/'

In [5]:
df_sigmoid_subt = pd.read_csv('../data/subtlety_playfight_data/curve_fit_subt/all_params_1k_subt_mixed.csv', index_col=[0])
df_sigmoid_subt.shape, df_sigmoid_subt.columns

((279, 11),
 Index(['r_squared', 'nrmse', 'aic', 'center', 'obj_center', 'sigma',
        'bias_xmin', 'bias_xmax', 'bias_lower', 'bias_upper', 'flipped'],
       dtype='object'))

In [6]:
bad_rows = (np.isnan(df_sigmoid_subt['obj_center'])) | \
      (np.isinf(df_sigmoid_subt['obj_center'])) | \
    (df_sigmoid_subt['obj_center']<0)

df_sigmoid_subt.loc[bad_rows,:]

,r_squared,nrmse,aic,center,obj_center,sigma,bias_xmin,bias_xmax,bias_lower,bias_upper,flipped
subID,,,,,,,,,,,
30153.0,0.493481,0.234090,-16.328702,0.196101,NaN,0.202573,0.635276,0.077545,0.523333,0.070000,False
30156.0,0.994711,0.029179,-45.480259,0.477303,NaN,0.096263,0.585949,0.043303,0.583333,0.041667,False
30201.0,0.992987,0.041195,-40.652173,0.418479,NaN,0.042199,0.561688,0.015000,0.561667,0.015000,False
30235.0,0.861362,0.130437,-24.516068,0.397311,NaN,0.110271,0.616466,0.086292,0.608333,0.085000,False
30246.0,0.925000,0.125988,-25.001943,0.491683,-inf,0.011999,0.500000,0.000000,0.500000,0.000000,False
30273.0,0.920315,0.103373,-27.771785,0.339515,NaN,0.130679,0.529537,0.120733,0.503333,0.118333,False
30297.0,0.701670,0.198372,-18.646576,0.586961,NaN,0.223000,0.645913,0.105600,0.625000,0.063333,False
30312.0,0.995885,0.027600,-46.259324,0.464891,NaN,0.068823,0.613706,0.066801,0.613333,0.066667,False
30342.0,0.811791,0.172931,-20.568092,0.435110,NaN,0.036179,0.526669,0.035000,0.526667,0.035000,False


In [7]:
df_sigmoid_subt = fix_obj_center(df_sigmoid_subt)

bad_rows = (np.isnan(df_sigmoid_subt['obj_center'])) | \
      (np.isinf(df_sigmoid_subt['obj_center'])) | \
    (df_sigmoid_subt['obj_center']<0)
    
df_sigmoid_subt.loc[bad_rows,:]

,r_squared,nrmse,aic,center,obj_center,sigma,bias_xmin,bias_xmax,bias_lower,bias_upper,flipped
subID,,,,,,,,,,,


In [8]:
df_sigmoid_subt = add_new_params(df_sigmoid_subt)
df_sigmoid_subt.loc[df_sigmoid_subt['range']<0,:]

,r_squared,nrmse,aic,center,obj_center,sigma,bias_xmin,bias_xmax,bias_lower,bias_upper,flipped,range,bias
subID,,,,,,,,,,,,,


In [9]:
df_sigmoid_subt.loc[np.isnan(df_sigmoid_subt['range']),:]

,r_squared,nrmse,aic,center,obj_center,sigma,bias_xmin,bias_xmax,bias_lower,bias_upper,flipped,range,bias
subID,,,,,,,,,,,,,


In [10]:
df_sigmoid_subt.loc[np.isnan(df_sigmoid_subt['bias']),:]

,r_squared,nrmse,aic,center,obj_center,sigma,bias_xmin,bias_xmax,bias_lower,bias_upper,flipped,range,bias
subID,,,,,,,,,,,,,


In [11]:
df_sigmoid_pf = pd.read_csv('../data/subtlety_playfight_data/curve_fit_pf/all_params_1k_pf_mixed.csv', index_col=[0])
bad_rows = (np.isnan(df_sigmoid_pf['obj_center'])) | \
      (np.isinf(df_sigmoid_pf['obj_center'])) | \
    (df_sigmoid_pf['obj_center']<0)
df_sigmoid_pf.loc[bad_rows,:]

,r_squared,nrmse,aic,center,obj_center,sigma,bias_xmin,bias_xmax,bias_lower,bias_upper,flipped
subID,,,,,,,,,,,
30118.0,0.579173,0.205289,-18.166683,0.531223,NaN,0.315880,0.721374,0.091007,0.676667,0.038333,False
30124.0,0.036874,0.317171,-12.076381,0.400178,-0.054884,1.000000,0.503322,0.434777,0.401667,0.345000,False
30249.0,-0.263323,0.345520,-10.877871,1.068182,NaN,1.000000,0.672591,0.307338,0.650000,0.261667,False
30286.0,0.760591,0.160899,-21.577665,0.412686,NaN,0.247385,0.985978,0.001420,0.983333,0.000000,False
30321.0,0.267563,0.270904,-14.283865,0.527594,-0.065880,0.469725,0.487874,0.734089,0.610000,0.846000,True
30330.0,0.942743,0.106527,-27.351000,0.219257,NaN,0.048327,0.776695,0.065000,0.775000,0.065000,False


In [12]:
df_sigmoid_pf = fix_obj_center(df_sigmoid_pf)
bad_rows = (np.isnan(df_sigmoid_pf['obj_center'])) | \
      (np.isinf(df_sigmoid_pf['obj_center'])) | \
    (df_sigmoid_pf['obj_center']<0)
df_sigmoid_pf.loc[bad_rows,:]

,r_squared,nrmse,aic,center,obj_center,sigma,bias_xmin,bias_xmax,bias_lower,bias_upper,flipped
subID,,,,,,,,,,,
30063.0,0.840255,0.135085,-24.025918,0.355551,NaN,0.207596,0.757791,0.720743,0.783333,0.811667,True
30227.0,0.916173,0.106480,-27.357175,0.305133,NaN,0.143479,0.667625,0.789959,0.671667,0.845000,True
30321.0,0.267563,0.270904,-14.283865,0.527594,NaN,0.469725,0.487874,0.734089,0.610000,0.846000,True


In [13]:
df_sigmoid_pf = add_new_params(df_sigmoid_pf)
df_sigmoid_pf.loc[df_sigmoid_pf['range']<0,:]

,r_squared,nrmse,aic,center,obj_center,sigma,bias_xmin,bias_xmax,bias_lower,bias_upper,flipped,range,bias
subID,,,,,,,,,,,,,
30063.0,0.840255,0.135085,-24.025918,0.355551,NaN,0.207596,0.757791,0.720743,0.783333,0.811667,True,-0.478533,0.512529
30227.0,0.916173,0.106480,-27.357175,0.305133,NaN,0.143479,0.667625,0.789959,0.671667,0.845000,True,-0.457585,0.458036
30321.0,0.267563,0.270904,-14.283865,0.527594,NaN,0.469725,0.487874,0.734089,0.610000,0.846000,True,-0.221963,0.399254


In [14]:
df_sigmoid_pf = df_sigmoid_pf.loc[df_sigmoid_pf['range']>0,:] # remove negative amplitudes
bad_rows = (np.isnan(df_sigmoid_pf['obj_center'])) | \
      (np.isinf(df_sigmoid_pf['obj_center'])) | \
    (df_sigmoid_pf['obj_center']<0)
df_sigmoid_pf.loc[bad_rows,:] # no more NaN obj_centers after that


,r_squared,nrmse,aic,center,obj_center,sigma,bias_xmin,bias_xmax,bias_lower,bias_upper,flipped,range,bias
subID,,,,,,,,,,,,,


In [15]:
df_sigmoid_pf.loc[np.isnan(df_sigmoid_pf['range']),:]

,r_squared,nrmse,aic,center,obj_center,sigma,bias_xmin,bias_xmax,bias_lower,bias_upper,flipped,range,bias
subID,,,,,,,,,,,,,


In [16]:
df_sigmoid_pf.loc[np.isnan(df_sigmoid_pf['bias']),:]

,r_squared,nrmse,aic,center,obj_center,sigma,bias_xmin,bias_xmax,bias_lower,bias_upper,flipped,range,bias
subID,,,,,,,,,,,,,


In [17]:
df_sigmoid_subt.rename(columns={'obj_center':'PSE'},inplace=True)
df_sigmoid_pf.rename(columns={'obj_center':'PSE'},inplace=True)

In [18]:
def return_minmax(df,param):
    print(f'{param}: min={df[param].describe()["min"]:.2f}, max={df[param].describe()["max"]:.2f}')

In [19]:
return_minmax(df_sigmoid_subt,'PSE')
return_minmax(df_sigmoid_subt,'range')
return_minmax(df_sigmoid_subt,'bias')

return_minmax(df_sigmoid_pf,'PSE')
return_minmax(df_sigmoid_pf,'range')
return_minmax(df_sigmoid_subt,'bias')

PSE: min=0.00, max=0.73
range: min=0.20, max=1.00
bias: min=0.11, max=1.00
PSE: min=0.00, max=0.96
range: min=0.01, max=1.00
bias: min=0.11, max=1.00


In [20]:
np.where(np.isnan(df_sigmoid_subt))

(array([], dtype=int64), array([], dtype=int64))

In [21]:
np.where(np.isnan(df_sigmoid_pf))

(array([], dtype=int64), array([], dtype=int64))

In [22]:
both_expts = df_sigmoid_subt.merge(df_sigmoid_pf,how='inner',on='subID',suffixes = ['_subt','_pf'])
both_expts.head()

,r_squared_subt,nrmse_subt,aic_subt,center_subt,PSE_subt,sigma_subt,bias_xmin_subt,bias_xmax_subt,bias_lower_subt,bias_upper_subt,...,center_pf,PSE_pf,sigma_pf,bias_xmin_pf,bias_xmax_pf,bias_lower_pf,bias_upper_pf,flipped_pf,range_pf,bias_pf
subID,,,,,,,,,,,,,,,,,,,,,
30002.0,0.978946,0.059585,-35.485036,0.546183,0.552951,0.103360,0.031291,0.067887,0.026667,0.056667,...,0.665858,0.591039,0.142704,0.212395,0.071276,0.205000,0.001667,False,0.716329,0.748738
30004.0,0.994784,0.031310,-44.493407,0.311200,0.276896,0.076866,0.194061,0.000105,0.180000,0.000000,...,0.522624,0.136066,0.224573,0.466189,0.100732,0.418333,0.043333,False,0.433079,0.822317
30005.0,0.985636,0.048320,-38.418617,0.380063,0.320583,0.115353,0.249789,0.040080,0.223333,0.036667,...,0.571110,0.530726,0.203235,0.162070,0.121117,0.113333,0.028333,False,0.716813,0.572306
30006.0,0.983710,0.057399,-36.008190,0.465887,0.464823,0.079221,0.009433,0.001171,0.006667,0.000000,...,0.263991,0.228279,0.167483,0.270824,0.050142,0.128333,0.040000,False,0.679035,0.843779
30008.0,0.900792,0.110236,-26.871886,0.385415,0.325118,0.172585,0.289378,0.128370,0.225000,0.110000,...,0.453739,0.340016,0.189201,0.362350,0.198502,0.320000,0.171667,False,0.439149,0.646071


In [23]:
both_expts.columns

Index(['r_squared_subt', 'nrmse_subt', 'aic_subt', 'center_subt', 'PSE_subt',
       'sigma_subt', 'bias_xmin_subt', 'bias_xmax_subt', 'bias_lower_subt',
       'bias_upper_subt', 'flipped_subt', 'range_subt', 'bias_subt',
       'r_squared_pf', 'nrmse_pf', 'aic_pf', 'center_pf', 'PSE_pf', 'sigma_pf',
       'bias_xmin_pf', 'bias_xmax_pf', 'bias_lower_pf', 'bias_upper_pf',
       'flipped_pf', 'range_pf', 'bias_pf'],
      dtype='object')

# merge traits across mini-batches

In [24]:
def merge_dfs(trait):

    root_folder = '../data/subtlety_playfight_data/traits/'
    # merge traits from the 2 batches
    if trait not in ['ffi','ucla','friends']:
        trait_batch1 = pd.read_csv(f'{root_folder}/traits_first50/{trait}_missResp.csv', index_col=[0])
        trait_batch2 = pd.read_csv(f'{root_folder}/traits_allelse/{trait}_missResp.csv', index_col=[0])
    elif trait == 'friends':
        trait_batch1 = pd.read_csv(f'{root_folder}/traits_first50/{trait}_onlygood.csv', index_col=[0])
        trait_batch2 = pd.read_csv(f'{root_folder}/traits_allelse/{trait}_onlygood.csv', index_col=[0])
    else:
        trait_batch1 = pd.read_csv(f'{root_folder}/traits_first50/{trait}_missResp.csv')
        trait_batch2 = pd.read_csv(f'{root_folder}/traits_allelse/{trait}_missResp.csv')
    trait_batch1.rename(columns={'ParticipantID':'subID'},inplace=True)
    trait_batch2.rename(columns={'ParticipantID':'subID'},inplace=True)
    df_traits_merged = pd.concat([trait_batch1.set_index('subID'),trait_batch2.set_index('subID')]).reset_index()
    print(trait, '. shape:', trait_batch1.shape, trait_batch2.shape,df_traits_merged.shape)

    return df_traits_merged


In [25]:
df_aq = merge_dfs('aq')
cols = ['subID','social_skill_reweigh',	'attn_switch_reweigh', 'img_reweigh', 'attn_to_det_reweigh','comm_reweigh']
df_aq = df_aq[cols]
new_cols = [i[:-8] if i!='subID' else i for i in cols]
df_aq.rename(columns=dict(zip(cols,new_cols)),inplace=True)
df_aq.head()

aq . shape: (34, 73) (245, 73) (279, 73)


,subID,social_skill,attn_switch,img,attn_to_det,comm
0,30002,21.0,23.0,9.0,10.0,16.000000
1,30004,21.0,20.0,9.0,15.0,15.000000
2,30005,17.0,29.0,21.0,16.0,16.000000
3,30006,8.0,10.0,5.0,10.0,3.000000
4,30008,5.0,15.0,12.0,17.0,12.222222


In [26]:
df_panas = merge_dfs('panas')
cols = ['subID','Positive_reweigh', 'Negative_reweigh']
df_panas = df_panas[cols]
new_cols = ['subID','posAffect','negAffect'] #[i[:-8] if i!='subID' else i for i in cols]
df_panas.rename(columns=dict(zip(cols,new_cols)),inplace=True)
df_panas.head()

panas . shape: (34, 31) (245, 31) (279, 31)


,subID,posAffect,negAffect
0,30002,20.0,0.0
1,30004,25.0,11.0
2,30005,20.0,21.0
3,30006,25.0,0.0
4,30008,28.0,4.0


In [27]:
df_ffi = merge_dfs('ffi')
cols = ['subID', 'neuroticism_reweigh', 'extraversion_reweigh', 'openness_reweigh',	'agreeableness_reweigh', 'conscientiousness_reweigh']
df_ffi = df_ffi[cols]
new_cols = [i[:-8] if i!='subID' else i for i in cols]
df_ffi.rename(columns=dict(zip(cols,new_cols)),inplace=True)
df_ffi.head()

ffi . shape: (34, 85) (245, 85) (279, 85)


,subID,neuroticism,extraversion,openness,agreeableness,conscientiousness
0,30002,38.0,28.000000,31.000000,34.0,39.0
1,30004,38.0,26.181818,50.000000,50.0,46.0
2,30005,51.0,39.000000,40.000000,43.0,19.0
3,30006,18.0,41.000000,31.000000,44.0,57.0
4,30008,27.0,46.000000,42.545455,46.0,50.0


In [28]:
df_ucla = merge_dfs('ucla')
cols = ['subID', 'total_reweigh']
df_ucla = df_ucla[cols]
df_ucla.rename(columns = {'total_reweigh':'loneliness'},inplace=True)
df_ucla.head()

ucla . shape: (34, 26) (245, 26) (279, 26)


,subID,loneliness
0,30002,56.0
1,30004,56.0
2,30005,41.0
3,30006,30.0
4,30008,45.0


In [29]:
df_fr = merge_dfs('friends')
cols = ['subID', 'nfriends']
df_fr = df_fr[cols]
df_fr.head()

friends . shape: (34, 2) (245, 2) (279, 2)


,subID,nfriends
0,30002,3.0
1,30004,3.0
2,30005,5.0
3,30006,4.0
4,30008,5.0


In [30]:
len(np.where(both_expts.reset_index()['subID']<30050)[0])# sanity check about the 2 batches

34

In [31]:
np.where(np.isnan(both_expts))

(array([], dtype=int64), array([], dtype=int64))

In [32]:
both_expts.columns

Index(['r_squared_subt', 'nrmse_subt', 'aic_subt', 'center_subt', 'PSE_subt',
       'sigma_subt', 'bias_xmin_subt', 'bias_xmax_subt', 'bias_lower_subt',
       'bias_upper_subt', 'flipped_subt', 'range_subt', 'bias_subt',
       'r_squared_pf', 'nrmse_pf', 'aic_pf', 'center_pf', 'PSE_pf', 'sigma_pf',
       'bias_xmin_pf', 'bias_xmax_pf', 'bias_lower_pf', 'bias_upper_pf',
       'flipped_pf', 'range_pf', 'bias_pf'],
      dtype='object')

# Merge trait and behavior

In [33]:
both_expts.shape, df_aq.shape, df_panas.shape, df_ffi.shape, df_ucla.shape, df_fr.shape # diff is because 3 neg amp subs were removed

((276, 26), (279, 6), (279, 3), (279, 6), (279, 2), (279, 2))

In [34]:
index_col = 'subID'
df_merged = pd.concat([both_expts,
    df_aq.set_index(index_col), \
    df_panas.set_index(index_col), \
    df_ffi.set_index(index_col), \
    df_ucla.set_index(index_col), \
        df_fr.set_index(index_col)],axis = 1,join='inner').reset_index()
df_merged.rename(columns = {'index':index_col},inplace=True)
df = df_merged.reset_index()
print(df.shape)
df.head()

(276, 42)


,index,subID,r_squared_subt,nrmse_subt,aic_subt,center_subt,PSE_subt,sigma_subt,bias_xmin_subt,bias_xmax_subt,...,comm,posAffect,negAffect,neuroticism,extraversion,openness,agreeableness,conscientiousness,loneliness,nfriends
0,0,30002.0,0.978946,0.059585,-35.485036,0.546183,0.552951,0.103360,0.031291,0.067887,...,16.000000,20.0,0.0,38.0,28.000000,31.000000,34.0,39.0,56.0,3.0
1,1,30004.0,0.994784,0.031310,-44.493407,0.311200,0.276896,0.076866,0.194061,0.000105,...,15.000000,25.0,11.0,38.0,26.181818,50.000000,50.0,46.0,56.0,3.0
2,2,30005.0,0.985636,0.048320,-38.418617,0.380063,0.320583,0.115353,0.249789,0.040080,...,16.000000,20.0,21.0,51.0,39.000000,40.000000,43.0,19.0,41.0,5.0
3,3,30006.0,0.983710,0.057399,-36.008190,0.465887,0.464823,0.079221,0.009433,0.001171,...,3.000000,25.0,0.0,18.0,41.000000,31.000000,44.0,57.0,30.0,4.0
4,4,30008.0,0.900792,0.110236,-26.871886,0.385415,0.325118,0.172585,0.289378,0.128370,...,12.222222,28.0,4.0,27.0,46.000000,42.545455,46.0,50.0,45.0,5.0


In [35]:
np.where(np.isnan(df_aq))

(array([], dtype=int64), array([], dtype=int64))

In [36]:
np.where(np.isnan(df_panas))

(array([], dtype=int64), array([], dtype=int64))

In [37]:
np.where(np.isnan(df_ffi))

(array([], dtype=int64), array([], dtype=int64))

In [38]:
np.where(np.isnan(df_ucla))

(array([], dtype=int64), array([], dtype=int64))

In [39]:
np.where(np.isnan(df_fr))

(array([62]), array([1]))

In [40]:
# df.to_csv('../data/subtlety_playfight_data/curvefit_traits_merged.csv')
# df.head()

In [41]:
print(df.shape)
df_nonan = df.dropna()#inplace=True)
print(df.shape,df_nonan.shape)

(276, 42)
(276, 42) (275, 42)


In [42]:
df.to_csv('../data/subtlety_playfight_data/curvefit_traits_merged.csv')
df.head()

,index,subID,r_squared_subt,nrmse_subt,aic_subt,center_subt,PSE_subt,sigma_subt,bias_xmin_subt,bias_xmax_subt,...,comm,posAffect,negAffect,neuroticism,extraversion,openness,agreeableness,conscientiousness,loneliness,nfriends
0,0,30002.0,0.978946,0.059585,-35.485036,0.546183,0.552951,0.103360,0.031291,0.067887,...,16.000000,20.0,0.0,38.0,28.000000,31.000000,34.0,39.0,56.0,3.0
1,1,30004.0,0.994784,0.031310,-44.493407,0.311200,0.276896,0.076866,0.194061,0.000105,...,15.000000,25.0,11.0,38.0,26.181818,50.000000,50.0,46.0,56.0,3.0
2,2,30005.0,0.985636,0.048320,-38.418617,0.380063,0.320583,0.115353,0.249789,0.040080,...,16.000000,20.0,21.0,51.0,39.000000,40.000000,43.0,19.0,41.0,5.0
3,3,30006.0,0.983710,0.057399,-36.008190,0.465887,0.464823,0.079221,0.009433,0.001171,...,3.000000,25.0,0.0,18.0,41.000000,31.000000,44.0,57.0,30.0,4.0
4,4,30008.0,0.900792,0.110236,-26.871886,0.385415,0.325118,0.172585,0.289378,0.128370,...,12.222222,28.0,4.0,27.0,46.000000,42.545455,46.0,50.0,45.0,5.0


In [43]:
df.columns

Index(['index', 'subID', 'r_squared_subt', 'nrmse_subt', 'aic_subt',
       'center_subt', 'PSE_subt', 'sigma_subt', 'bias_xmin_subt',
       'bias_xmax_subt', 'bias_lower_subt', 'bias_upper_subt', 'flipped_subt',
       'range_subt', 'bias_subt', 'r_squared_pf', 'nrmse_pf', 'aic_pf',
       'center_pf', 'PSE_pf', 'sigma_pf', 'bias_xmin_pf', 'bias_xmax_pf',
       'bias_lower_pf', 'bias_upper_pf', 'flipped_pf', 'range_pf', 'bias_pf',
       'social_skill', 'attn_switch', 'img', 'attn_to_det', 'comm',
       'posAffect', 'negAffect', 'neuroticism', 'extraversion', 'openness',
       'agreeableness', 'conscientiousness', 'loneliness', 'nfriends'],
      dtype='object')

In [44]:
onlynec_cols = ['subID','PSE_subt','range_subt','bias_subt','PSE_pf','range_pf','bias_pf','social_skill','comm','loneliness','posAffect']
df = df[onlynec_cols]
df.to_csv('../data/subtlety_playfight_data/curvefit_traits_merged_onlyneccols.csv')